In [1]:
import re
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [2]:
# global variable initialization
YEARS_TO_COUNT = ['1990', '1991', '1992', '1993','1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

Crawl data from NOAA site, load CSV files to data frame and organize the relevant data.

In [ ]:
# utility functions for data frame manipulation
def convert_miles_to_km(miles):
    return round(miles * 1.60934, 2)

def convert_yards_to_meters(yards):
    return round(yards * 0.9144, 2)

def create_date_in_df(df):
    df.BEGIN_YEARMONTH = df.BEGIN_YEARMONTH.astype(str)
    df.BEGIN_DAY = df.BEGIN_DAY.astype(str)
    df['Year'] = df.BEGIN_YEARMONTH.str[:4]
    df['Month'] = df.BEGIN_YEARMONTH.str[4:]
    df['Date'] = df.iloc[:,1].str.cat(df.iloc[:,13], sep='/').str.cat(df.iloc[:,12], sep='/')
    df.drop(columns=['BEGIN_YEARMONTH', 'BEGIN_DAY', 'Month', 'Year'], inplace=True)
    return df

In [ ]:
# functions for csv scraping
def getCSVSoup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

def get_csv_url_from_soup(soup,year):
    return soup.select(f'a[href*="details-ftp_v1.0_d{year}"]')[0]['href']

def get_df_bs_crawler():
    noaa_csv_url = 'https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/'
    col_list = ["BEGIN_YEARMONTH", "BEGIN_DAY", "STATE", "EVENT_TYPE", "CZ_NAME", "BEGIN_DATE_TIME", "DEATHS_DIRECT", "TOR_F_SCALE", "TOR_LENGTH", "TOR_WIDTH", "BEGIN_LOCATION", "BEGIN_LAT", "BEGIN_LON"]
    df_list = []
    for year in YEARS_TO_COUNT:
        soup = getCSVSoup(noaa_csv_url)
        csv_url = get_csv_url_from_soup(soup, year)
        df_list.append(pd.read_csv(f'{noaa_csv_url}{csv_url}', compression='gzip', usecols=col_list))
    df = pd.concat(df_list, ignore_index=True)
    return df

def clean_df_csv(df):
    rename_dict = {"STATE": "Country","CZ_NAME":"District", "TOR_F_SCALE": "Scale", "TOR_LENGTH":"Length (KM)","TOR_WIDTH":"Width (M)","BEGIN_LOCATION":"City","BEGIN_LAT":"Latitude","BEGIN_LON":"Longtitude", "DEATHS_DIRECT": "Deaths", "YEAR": "Year"}
    df.rename(columns=rename_dict, inplace=True)
    split_df = df.BEGIN_DATE_TIME.str.split(' ', expand=True)
    df = df[df['EVENT_TYPE'] == 'Tornado'].drop(columns=['EVENT_TYPE', "BEGIN_DATE_TIME"])
    df['Time'] = split_df[1].str[:5]
    df = create_date_in_df(df)
    df['Length (KM)'] = df['Length (KM)'].apply(lambda x: convert_miles_to_km(x))
    df["Width (M)"] = df['Width (M)'].apply(lambda x: convert_yards_to_meters(x))
    df = df[["District","City","Country","Longtitude", "Latitude" ,"Date","Time","Scale","Length (KM)", "Width (M)", "Deaths"]]
    df.reset_index(drop=True, inplace=True)
    return df

In [ ]:
def merge_csvs():
    noaadf = pd.read_csv('noaa_data.csv')
    eswddf = pd.read_csv('eswd_data.csv')
    list = [noaadf, eswddf]
    df = pd.concat(list, ignore_index=True)
    df.to_csv('merged_data.csv', index=False)


In [ ]:
# initiate csv crawler
def run_csv_crawl():
    start_csv_xpath = "//a[contains(@href, 'details-ftp_v1.0_d"
    end_csv_xpath = "')]"
    rename_dict = {"STATE": "Country","CZ_NAME":"District", "TOR_F_SCALE": "Scale", "TOR_LENGTH":"Length","TOR_WIDTH":"Width","BEGIN_LOCATION":"City","BEGIN_LAT":"Latitude","BEGIN_LON":"Longtitude", "DEATHS_DIRECT": "Deaths", "YEAR": "Year"}
    col_list = ["BEGIN_YEARMONTH", "BEGIN_DAY", "STATE", "EVENT_TYPE", "CZ_NAME", "BEGIN_DATE_TIME", "DEATHS_DIRECT", "TOR_F_SCALE", "TOR_LENGTH", "TOR_WIDTH", "BEGIN_LOCATION", "BEGIN_LAT", "BEGIN_LON"]
    df_list = []
    csv_link = "https://www1.ncdc.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
    csv_driver = create_web_driver()
    start_web_driver(csv_driver, csv_link)
    time.sleep(1)
    for i in range(1994,2018):
        df_to_append = pd.read_csv(csv_link + csv_driver.find_element_by_xpath(f"{start_csv_xpath}{i}{end_csv_xpath}").text, compression='gzip', usecols=col_list)
        df_to_append.rename(columns=rename_dict, inplace=True)
        split_df = df_to_append.BEGIN_DATE_TIME.str.split(' ', expand=True)
        df_to_append = df_to_append[df_to_append['EVENT_TYPE'] == 'Tornado'].drop(columns=['EVENT_TYPE', "BEGIN_DATE_TIME"])
        df_to_append['Time'] = split_df[1].str[:5]
        df_to_append = create_date_in_df(df_to_append)
        df_to_append["Length"] = df_to_append.Length.apply(lambda x: convert_miles_to_km(x))
        df_to_append["Width"] = df_to_append.Width.apply(lambda x: convert_yards_to_meters(x))
        df_list.append(df_to_append)
    csv_df = pd.concat(df_list, ignore_index=True)
    csv_df = csv_df[["District","City","Country","Longtitude", "Latitude" ,"Date","Time","Scale","Length", "Width", "Deaths"]]
    csv_df.to_csv('noaa_data.csv')

In [ ]:
def run_noaa_crawl():
    noaadf = get_df_bs_crawler()
    noaadf = clean_df_csv(noaadf)
    noaadf.to_csv('noaa_data.csv', index=False)

Run web crawler on ESWD site, scrape each query for relevant data, and put it in a dataframe for further analysis later.

In [ ]:
# auxilary functions for creating the web driver and closing it
def create_web_driver():
    return webdriver.Chrome(ChromeDriverManager().install())

def start_web_driver(driver, url):
    driver.maximize_window()
    driver.get(url)

def close_web_driver(driver):
    driver.close()

In [ ]:
# global variable initialization for ESWD
MONTHS_TO_COUNT = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
ESWD_URL = 'https://eswd.eu/cgi-bin/eswd.cgi'
ESWD_TORNADO_XPATH = '//*[@name="TORNADO"]'
START_DATE_XPATH = '//*[@id="start_date"]'
END_DATE_XPATH = '//*[@id="end_date"]'
FIND_REPORTS_COUNT_XPATH = "//p[contains(text(),'selected reports')] | //p[contains(text(),'no reports')]"
SUBMIT_XPATH = '(//*[@value="submit query"])[2]'

In [ ]:
# ESWD auxilary functions for date parsing
def get_end_date(curr_year, curr_month):
    if curr_month == '02':
        day = '28'
    elif curr_month in ['04','06', '09','11']:
        day = '30'
    else:
        day = '31'
    return f'{day}-{curr_month}-{curr_year}'

def get_start_date(curr_year,curr_month):
    return f'01-{curr_month}-{curr_year}'

def input_date(driver, start_date, end_date, start_date_xpath, end_date_xpath):
    driver.find_element_by_xpath(start_date_xpath).clear()
    time.sleep(0.5)
    driver.find_element_by_xpath(start_date_xpath).send_keys(start_date)
    driver.find_element_by_xpath(end_date_xpath).clear()
    time.sleep(0.5)
    driver.find_element_by_xpath(end_date_xpath).send_keys(end_date)


In [ ]:
# function to accumulate the data from the site in lists to prepare for dataframe creation using other auxilary functions
def get_data_from_page(driver,district, city, country, long, lat, date, times, scale,length, width,deaths):
    base_xpath ="//td[@class='base_info']/p[b and not(@class='smallgray')]"
    district_xpath = "/b[1]"
    bold_tags_xpath = "/b"
    country_xpath = "/b[2]"
    date_xpath = "/b[3]"
    time_xpath = "/b[4]"
    scale_xpath_1 = "(//p[@class='TORNADO detail_info_entry'])"
    scale_xpath_2 = "/b[contains(text(),'F')]" 
    scale_xpath_3 = "((//p[@class='TORNADO'][contains(b,'tornado')])"
    scale_xpath_4 = "/b[text()='tornado'])"
    reports_count = len(driver.find_elements_by_xpath(base_xpath))
    len_elements = driver.find_elements_by_xpath(scale_xpath_1)
    parent_items= driver.find_elements_by_xpath(base_xpath)
    for i in range(reports_count):
        district = get_elements_from_list(driver,district_xpath,base_xpath,district,i)
        child_item = driver.find_elements_by_xpath((f'({base_xpath})[{i+1}]{bold_tags_xpath}'))
        child_item_text = get_text_from_element_list(child_item)
        city= get_city_list(child_item_text,parent_items[i].text,city,0)
        country = get_elements_from_list(driver,country_xpath,base_xpath,country,i)
        long, lat =  get_long_lat_lists(child_item_text,parent_items[i].text,long,lat,1)
        date = get_elements_from_list(driver,date_xpath,base_xpath,date,i)
        times = get_elements_from_list(driver,time_xpath,base_xpath,times,i)
        scale_elements_list = driver.find_elements_by_xpath((f'{scale_xpath_1}[{i+1}]{scale_xpath_2} | {scale_xpath_3}[{i+1}]{scale_xpath_4}'))
        scale = get_scale_list(scale,scale_elements_list)
        length = get_len_list(child_item_text,parent_items[i].text,length,1,len_elements[i])
        width = get_width_list(width, len_elements[i])
        deaths = get_deaths_list(deaths,len_elements[i])
    return district, city, country, long,lat, date, times, scale,length,width,deaths


In [ ]:
# auxilary functions in charge of initial parsing and cleaning of the data captured by XPATH
def replace_child_text(child_text,field):
    for word in child_text.split(' '):
        field = field.replace('\n','').replace(word,'',1).lstrip()
    return field

def checks_split_position(field,selector):
    if field[0] == '(':
        selector = selector + 1
    return selector

def checks_digit(field,selector):
    return field.split('(')[selector][0].isdigit()

def checks_len(field):
    return '<' in field
    
def get_text_from_element_list(elem_list):
    text = ''
    for elem in elem_list:
        text += elem.text + ' '
    return text

In [ ]:
# auxilary functions to clean the data further and appending it to the relevant lists
def convert_w_e_to_long(longtitude):
    if 'W' in longtitude:
        longtitude = longtitude.replace('W','').strip()
        longtitude = -float(longtitude)
    elif 'E' in longtitude:
        longtitude = longtitude.replace('E','').strip()
        longtitude = float(longtitude)
    return longtitude

def convert_n_to_lat(latitude):
    latitude = latitude.replace('N','').strip()
    latitude = float(latitude)
    return latitude

def append_coordinates_lists(field,long,lat,position):
    regexp = re.compile(r'[0-9]*\.[0-9]+ [a-zA-Z]+')
    if checks_len(field):
        coordinates = field.split('(')[position].split(')')[0].rstrip()
    else:
        coordinates = field.split('(')[position].rstrip().replace(')','')
    if not re.match(regexp,coordinates):
        long.append(np.nan)
        lat.append(np.nan)
    else:
        longtitude = coordinates.split(',')[1]
        longtitude = convert_w_e_to_long(longtitude)
        latitude = coordinates.split(',')[0]
        latitude = convert_n_to_lat(latitude)
        long.append(longtitude)
        lat.append(latitude)
    return long,lat

def get_long_lat_lists(child_text,field,long,lat,selector):
    field = replace_child_text(child_text,field)
    position = checks_split_position(field, selector)
    if checks_digit(field,position):
            long,lat = append_coordinates_lists(field,long,lat,position)
    else:
        if checks_digit(field,position-1):
            long,lat = append_coordinates_lists(field,long,lat,position-1)
        else:
            long.append(np.nan)
            lat.append(np.nan)
    return long,lat


In [ ]:
def get_deaths_list(curr_list, element):
    if 'dead:' in element.text:
        curr_list.append(element.text.split('dead:')[1].split('.')[0])
    else:
        curr_list.append(np.nan)
    return curr_list

In [ ]:
# auxilary functions to clean the data further and appending it to the relevant lists
'''if the size of the elements list is 1 it gets only "tornado" dummy text, so there is no scale in the text, otherwise there is scale 
in the text so take the relevant F scale'''
def get_scale_list(scale,scale_elements_list):
    if len(scale_elements_list) == 1:
        scale.append(np.nan)
    else:
        for element in scale_elements_list:
            if 'F' in element.text:
                scale.append(element.text)
    return scale

def get_width_list(curr_list, element):
    if 'path width:' in element.text:
        curr_list.append(element.text.split('path width:')[1].split('m')[0].strip()) 
    else:
        curr_list.append(np.nan)
    return curr_list

'''if there's a field in the right textbox with specified path length get the number presented in it, otherwise check if the first char is a digit,
 if so check if the field on the left has the "<" char, which indicates length, if so take it, otherwise input nan in the list'''
def get_len_list(child_text,field,curr_list,selector,element):
    if 'path length:' in element.text:
        curr_list.append(element.text.split('path length:')[1].split('k')[0].strip()) 
    else:
        field = replace_child_text(child_text,field)
        if checks_digit(field,selector):
            if checks_len(field):
                curr_list.append(field.split('<')[1].split('k')[0].strip()) 
            else:
                curr_list.append(np.nan)
        else:
            curr_list.append(np.nan)
    return curr_list

'''if the first character is a digit put nan, otherwise take the city name'''
def get_city_list(child_text,field,curr_list,selector):
    field = replace_child_text(child_text,field)
    position = checks_split_position(field, selector)
    if checks_digit(field,position):
        curr_list.append(np.nan)
    else:
        curr_list.append(field.split('(')[position].rstrip().replace(')',''))
    return curr_list

In [ ]:
#auxilary functions to input bold text to the relevant lists and create a df using the lists
def get_elements_from_list(driver,curr_xpath,base_xpath,curr_list,i):
    curr_list.append(driver.find_elements_by_xpath((base_xpath+curr_xpath))[i].text)
    return curr_list

def get_df_from_lists(district, city, country, long, lat, date, times, scale,length,width,deaths):
    df = pd.DataFrame({'District':district,'City':city,'Country':country,'Longtitude':long,'Latitude':lat, 'Date':date,'Time':times,'Scale':scale,'Length (KM)':length,'Width (M)':width,'Deaths':deaths})
    return df


In [ ]:
# main function to crawl the data from the website and create a df
def run_eswd_crawler():
    driver = create_web_driver()
    start_web_driver(driver,ESWD_URL)
    driver.find_element_by_xpath(ESWD_TORNADO_XPATH).click()
    district, city, country, long, lat, date, times, scale,length, width,deaths = [], [], [], [], [], [], [], [],[],[],[]
    for year in YEARS_TO_COUNT:
        for month in MONTHS_TO_COUNT:
            end_date = get_end_date(year,month)
            start_date = get_start_date(year, month)
            input_date(driver, start_date, end_date,START_DATE_XPATH,END_DATE_XPATH)
            driver.find_element_by_xpath(SUBMIT_XPATH).click()
            reports_amount = driver.find_element_by_xpath(FIND_REPORTS_COUNT_XPATH).text
            if "no report" in reports_amount:
                continue
            district, city, country, long, lat, date, times, scale,length,width,deaths = get_data_from_page(driver,district, city, country, long, lat, date, times, scale,length,width,deaths)
            edf = get_df_from_lists(district, city, country, long, lat, date, times, scale,length,width,deaths)
    edf.to_csv(f'ESWD_data.csv', index=False)
    close_web_driver(driver)

In [ ]:
run_noaa_crawl()
run_eswd_crawler()
merge_csvs()